<a href="https://colab.research.google.com/github/mbuguajackson/vegetation_mapping/blob/main/ndvi/ndvi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-my-mbugua')

In [9]:
kenya = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017") \
           .filter(ee.Filter.eq('country_na', 'Kenya'))


In [10]:
sentinel = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(kenya) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()


In [14]:
# Calculate NDVI
ndvi = sentinel.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Clip to Kenya
ndvi_kenya = ndvi.clip(kenya)

# Visualization
Map = geemap.Map(center=[0.5, 37.5], zoom=6)
ndvi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green'],
}
Map.addLayer(ndvi_kenya, ndvi_vis, 'NDVI - Kenya')
Map.addLayer(kenya, {}, "Kenya boundary")
Map.to_html("kenya_ndvi_map.html")
